In [50]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [134]:
movies = pd.read_csv(Path("archive/movies.csv"))

In [136]:
print(movies.shape)

(58098, 3)


In [53]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [54]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [55]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [56]:
movies["year_of_release"] = 0

In [57]:
movies.head()

,movieId,title,genres,year_of_release
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0
4,5,Father of the Bride Part II (1995),Comedy,0


In [148]:
# seperating year of release from title
for idx in movies.index:
    title = movies.loc[idx,"title"]
    year_of_release = re.findall(r"\([0-9]\)", title)
    for year in year_of_release:
        if year_of_release is None:
            movies.loc[idx,"year_of_release"] = np.NaN
        else:
            movies.loc[idx,"year_of_release"] = year


In [149]:
movies.isnull().sum()

movieId              0
title                0
genres               0
year_of_release    327
dtype: int64

In [150]:
movies.loc[movies["year_of_release"].isnull()]

,movieId,title,genres,year_of_release
10023,32930,Category 6: Day of Destruction,Action|Drama,NaN
10613,40697,Babylon 5,Sci-Fi,NaN
15719,79607,"Millions Game, The (Das Millionenspiel)",Action|Drama|Sci-Fi|Thriller,NaN
17444,87442,"Bicycle, Spoon, Apple (Bicicleta, cullera, poma)",Documentary,NaN
22651,107434,Diplomatic Immunity (2009– ),Comedy,NaN
...,...,...,...,...
57504,192339,The Sign of Three,(no genres listed),NaN
57674,192829,Parwaaz Hai Junoon,Adventure|Children|Romance,NaN
57789,193149,¿Qué te juegas?,(no genres listed),NaN
57903,193443,Wall,(no genres listed),NaN


In [151]:
movies.loc[movies.title == "Trails (Veredas) (1978)"]

,movieId,title,genres,year_of_release
17343,87061,Trails (Veredas) (1978),(no genres listed),1978


In [152]:
movies_new = pd.DataFrame(columns=["movieId","title","genre","year_of_release"])
movies_new.head()

,movieId,title,genres,year_of_release


In [153]:
for idx in movies.index:
    genre_list = movies.loc[idx,"genres"].split("|")
    for genre in genre_list:
        temp_df = pd.DataFrame(
            [[movies.loc[idx,"movieId"], movies.loc[idx,"title"],genre,movies.loc[idx,"year_of_release"]]],
            columns=["movieId","title","genre","year_of_release"]
            )
        movies_new = pd.concat([movies_new, temp_df], ignore_index= True)

In [154]:
movies_new.head()

,movieId,title,genres,year_of_release
0,1,Toy Story (1995),Adventure,1995
1,1,Toy Story (1995),Animation,1995
2,1,Toy Story (1995),Children,1995
3,1,Toy Story (1995),Comedy,1995
4,1,Toy Story (1995),Fantasy,1995


In [155]:
movies_new.head()

,movieId,title,genres,year_of_release
0,1,Toy Story (1995),Adventure,1995
1,1,Toy Story (1995),Animation,1995
2,1,Toy Story (1995),Children,1995
3,1,Toy Story (1995),Comedy,1995
4,1,Toy Story (1995),Fantasy,1995


In [157]:
genre_set = set(movies_new["genres"].values)

In [158]:
genres = pd.DataFrame(columns=["genreId", "genre"])
i = 1
for genre in genre_set:
    temp_df = pd.DataFrame([[i, genre]], columns=["genreId", "genre"])
    genres = pd.concat([genres, temp_df],ignore_index = True)
    i += 1

In [162]:
movies_new.loc[movies_new["genre"] == "(no genres listed)"]

,movieId,title,genre,year_of_release
34699,83773,Away with Words (San tiao ren) (1999),(no genres listed),1999
34728,83829,Scorpio Rising (1964),(no genres listed),1964
35071,84768,Glitterbug (1994),(no genres listed),1994
35672,86493,"Age of the Earth, The (A Idade da Terra) (1980)",(no genres listed),1980
35973,87061,Trails (Veredas) (1978),(no genres listed),1978
...,...,...,...,...
106048,193815,Tempting Fate (1998),(no genres listed),1998
106074,193849,Rafinha Bastos: Ultimatum (2018),(no genres listed),2018
106080,193855,"Spring Night, Summer Night (1967)",(no genres listed),1967
106094,193870,Isabelle Eberhardt (1991),(no genres listed),1991


In [163]:
genres

,genreId,genre
0,1,Adventure
1,2,Action
2,3,IMAX
3,4,War
4,5,Film-Noir
5,6,Western
6,7,Documentary
7,8,Animation
8,9,Drama
9,10,Horror


In [164]:
movie_genre_join_table = pd.DataFrame(columns=["movieId","genreId"])
movie_genre_join_table

,movieId,genreId


7    8
Name: genreId, dtype: object